In [2]:
import time
import arcpy
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import csv
import sys
pd.set_option("display.max_columns", 100)

In [4]:
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_geofile'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

df_pts_den=pd.read_excel(r'E:\project_basedata\2023data_research\basedata\basic_point_density\point_density.xlsx')
ls_city=df_pts_den['city'].unique().tolist()
ls0=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_dbscan_result_filt')
out_lay = 'point_bj_hd_31'
start = time.time()

for i in ls_city:
    i_=i+'_'
    for j in ls0:
        if i_ in j and j.endswith('100_re_1_to_4.csv'):
            print(i,j)
            
             # 首先把csv文件转化成点文件
            input_file='E:\\project_basedata\\2023data_research\\basedata\\basic_dbscan_result_filt\\'+j
            arcpy.MakeXYEventLayer_management(table=input_file, 
                                              in_x_field='dianping_shops_2021.longitude', 
                                              in_y_field='dianping_shops_2021.latitude', 
                                              out_layer=out_lay, spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_31','point_sh_111')
            print('point done')
            
            # 筛选核心点
            in_file='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_111.shp'
            arcpy.MakeFeatureLayer_management(in_file,'point_sh_112')
            arcpy.SelectLayerByAttribute_management('point_sh_112','NEW_SELECTION','core_4 <> -1')
            arcpy.CopyFeatures_management('point_sh_112','point_sh_121')
            print('core point done')
            
            # 根据核心点的范围形成fishnet
            outFeatureClass='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_net.shp'
            inFeatures='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_121.shp'
            intersect_feature='INTERSECTFEATURE'
            use_page_unit='NO_USEPAGEUNIT'
            polygonWidth = '200 meters'
            polygonHeight= '200 meters'
            scale='1'
            label_from_origin='NO_LABELFROMORIGIN'
            arcpy.GridIndexFeatures_cartography(outFeatureClass,inFeatures,intersect_feature,
                                                use_page_unit,scale,polygonWidth,polygonHeight)
            print('fishnet done')
            
            # 将筛选出的点文件打上筛选出的面文件的标签。并输出csv
            outfc_1='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_corejoin.shp'
            arcpy.SpatialJoin_analysis(target_features=inFeatures,
                                       join_features=outFeatureClass,
                                       join_operation='JOIN_ONE_TO_ONE',
                                       out_feature_class=outfc_1,match_option='WITHIN')
            outpath='E:\\project_basedata\\2023data_research\\basedata\\basic_cell'
            outname=i+'_poi_tag.csv'
            fieldmappings = arcpy.FieldMappings()
            fieldmap1=arcpy.FieldMap()
            fieldmap2=arcpy.FieldMap()
            fieldmap1.addInputField(outfc_1,'dianping_s')
            fieldmap2.addInputField(outfc_1,'PageName')
            fieldmappings.addFieldMap(fieldmap1)
            fieldmappings.addFieldMap(fieldmap2)
            arcpy.TableToTable_conversion(in_rows=outfc_1,out_path=outpath,out_name=outname,field_mapping=fieldmappings)
            print('core point spatial join done')
            
            # 将筛选后的面文件转化为点
            outFeatureClass_pts='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_fishpts.shp'
            arcpy.FeatureVerticesToPoints_management(outFeatureClass,outFeatureClass_pts,'ALL')
            print('area to pts done')
            
            # 给每个点加上坐标
            arcpy.AddXY_management(outFeatureClass_pts)
            print('area to pts add xy done')
            
            # 输出点文件和打上面标签的csv
            arr = arcpy.da.FeatureClassToNumPyArray(outFeatureClass_pts,
                                                    field_names = ('FID','PageName','POINT_X','POINT_Y'))
            df_new_xy = pd.DataFrame(arr)
            save_file='E:\\project_basedata\\2023data_research\\basedata\\basic_cell\\'+i+'_fish_pts.csv'
            df_new_xy.to_csv(save_file,index=False,mode='w')
            print('area pts to csv done')
            end = time.time()
            runTime = end - start
            print('time:', runTime)

(u'chongqing', 'chongqing_100_re_1_to_4.csv')
point done
core point done
fishnet done
core point spatial join done
area to pts done
area to pts add xy done
area pts to csv done
('time:', 626.8699998855591)
(u'dalian', 'dalian_100_re_1_to_4.csv')
point done
core point done
fishnet done
core point spatial join done
area to pts done
area to pts add xy done
area pts to csv done
('time:', 782.771999835968)
(u'beijing', 'beijing_100_re_1_to_4.csv')
point done
core point done
fishnet done
core point spatial join done
area to pts done
area to pts add xy done
area pts to csv done
('time:', 1142.6729998588562)
(u'shanghai', 'shanghai_100_re_1_to_4.csv')
point done
core point done
fishnet done
core point spatial join done
area to pts done
area to pts add xy done
area pts to csv done
('time:', 1550.4470000267029)
(u'nanjing', 'nanjing_100_re_1_to_4.csv')
point done
core point done
fishnet done
core point spatial join done
area to pts done
area to pts add xy done
area pts to csv done
('time:', 1703

In [6]:
arcpy.da.FeatureClassToNumPyArray(outfc_1, field_names = ('dianping_s','PageName'))
outTable='E:\\project_basedata\\2023data_research\\basedata\\basic_cell\\'+i+'_poi_tag.csv'
outLocation
arcpy.TableToTable_conversion(outfc_1, outLocation, outTable, expression)

MemoryError: cannot allocate array memory

In [13]:
outpath='E:\\project_basedata\\2023data_research\\basedata\\basic_cell'
outname=i+'_poi_tag.csv'
fieldmappings = arcpy.FieldMappings()
fieldmap1=arcpy.FieldMap()
fieldmap2=arcpy.FieldMap()
fieldmap1.addInputField(outfc_1,'dianping_s')
fieldmap2.addInputField(outfc_1,'PageName')
fieldmappings.addFieldMap(fieldmap1)
fieldmappings.addFieldMap(fieldmap2)
arcpy.TableToTable_conversion(in_rows=outfc_1,out_path=outpath,out_name=outname,field_mapping=fieldmappings)

<Result 'E:\\project_basedata\\2023data_research\\basedata\\basic_cell\\chongqing_poi_tag.csv'>

In [2]:
arcpy.env.overwriteOutput = True # 覆盖已存在文件
arcpy.env.workspace = r'E:\project_basedata\2023data_research\basedata\basic_geofile'
coord_sys = arcpy.SpatialReference(4490)
arcpy.env.outputCoordinateSystem = arcpy.SpatialReference(4525)

df_pts_den=pd.read_excel(r'E:\project_basedata\2023data_research\basedata\basic_point_density\point_density.xlsx')
ls_city=df_pts_den['city'].unique().tolist()
ls0=os.listdir(r'E:\project_basedata\2023data_research\basedata\basic_dbscan_result_filt')
out_lay = 'point_bj_hd_31'
ls_city=['chongqing']
for i in ls_city:
    i_=i+'_'
    for j in ls0:
        if i_ in j and j.endswith('100_re_1_to_4.csv'):
            print(i,j)
            
             # 首先把csv文件转化成点文件
            input_file='E:\\project_basedata\\2023data_research\\basedata\\basic_dbscan_result_filt\\'+j
            arcpy.MakeXYEventLayer_management(table=input_file, 
                                              in_x_field='dianping_shops_2021.longitude', 
                                              in_y_field='dianping_shops_2021.latitude', 
                                              out_layer=out_lay, spatial_reference=coord_sys)
            arcpy.CopyFeatures_management('point_bj_hd_31','point_sh_111')
            
            # 筛选核心点
            in_file='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_111.shp'
            arcpy.MakeFeatureLayer_management(in_file,'point_sh_112')
            arcpy.SelectLayerByAttribute_management('point_sh_112','NEW_SELECTION','core_4 <> -1')
            arcpy.CopyFeatures_management('point_sh_112','point_sh_121')
            
            # 根据核心点的范围形成fishnet
            outFeatureClass='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_net.shp'
            inFeatures='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_121.shp'
            intersect_feature='INTERSECTFEATURE'
            use_page_unit='NO_USEPAGEUNIT'
            polygonWidth = '200 meters'
            polygonHeight= '200 meters'
            label_from_origin='NO_LABELFROMORIGIN'
            arcpy.GridIndexFeatures_cartography(outFeatureClass,inFeatures,intersect_feature,
                                                use_page_unit,polygonWidth,polygonHeight,label_from_origin)
            
            # 根据核心点的范围形成fishnet
            outFeatureClass='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_net.shp'
            template_file='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_121.shp'
            cellSizeWidth = '200'
            cellSizeHeight = '200'
            numRows =  '0'
            numColumns = '0'
            arcpy.CreateFishnet_management(out_feature_class=outFeatureClass,template=template_file,
                                           cell_width=cellSizeWidth,cell_height=cellSizeHeight,number_rows=numRows,
                                           number_columns=numColumns,labels='NO_LABELS',geometry_type='POLYGON')
            
            # 筛选与点有关的网面文件
            outfc='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_fishjoin.shp'
            targ_f='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_12.shp'
            fieldmappings = arcpy.FieldMappings()
            fieldmap.addInputField(targ_f,'Id')
            fieldmappings.addFieldMap(fieldmap)
            arcpy.SpatialJoin_analysis(target_features=outFeatureClass,
                                       join_features=targ_f,
                                       join_operation='JOIN_ONE_TO_ONE',join_type='KEEP_COMMON',
                                       field_mapping=fieldmappings,
                                       out_feature_class=outfc,match_option='COMPLETELY_CONTAINS')
            
            # 将筛选出的点文件打上筛选出的面文件的标签。并输出csv
            outfc_1='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_corejoin.shp'
            targ_f='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\point_sh_12.shp'
            arcpy.SpatialJoin_analysis(target_features=targ_f,
                                       join_features=outfc,
                                       join_operation='JOIN_ONE_TO_ONE',
                                       out_feature_class=outfc_1,match_option='COMPLETELY_CONTAINS')
            arr = arcpy.da.FeatureClassToNumPyArray(outfc_1, field_names = ('dianping_s','TARGET_FID_1'))
            df_new_xy = pd.DataFrame(arr)
            save_file='E:\\project_basedata\\2023data_research\\basedata\\basic_cell\\'+i+'_poi_tag.csv'
            df_new_xy.to_csv(save_file,index=False,mode='w')
            
            # 将筛选后的面文件转化为点
            outFeatureClass='E:\\project_basedata\\2023data_research\\basedata\\basic_geofile\\'+i+'_fishpts.shp'
            arcpy.FeatureVerticesToPoints_management(outfc, outFeatureClass, 'ALL')
            
            # 给每个点加上坐标
            arcpy.AddXY_management(outFeatureClass)
            
            # 输出点文件和打上面标签的csv
            arr = arcpy.da.FeatureClassToNumPyArray(outFeatureClass, field_names = ('FID','TARGET_FID','POINT_X','POINT_Y'))
            df_new_xy = pd.DataFrame(arr)
            save_file='E:\\project_basedata\\2023data_research\\basedata\\basic_cell\\'+i+'_fish_pts.csv'
            df_new_xy.to_csv(save_file,index=False,mode='w')

('chongqing', 'chongqing_100_re_1_to_4.csv')


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000735: Fishnet Origin Coordinate : Value is required
ERROR 000735: Y-Axis Coordinate : Value is required
Failed to execute (CreateFishnet).


In [15]:
fieldmappings = arcpy.FieldMappings()
fieldmap=arcpy.FieldMap()
file1='E:\\project_basedata\\2023data_research\\basedata\\basic_test\\cq_fish_pts3.shp'
fieldmap.addInputField(file1,'Id')
fieldmappings.addFieldMap(fieldmap)

In [14]:
fieldmappings

<FieldMappings object at 0x19d871b0[0x19e8a6c8]>